# Chap10 时序数据

In [1]:
import numpy as np
import pandas as pd

## 时序中的基本对象

In [3]:
ts_dict = {'概念':['时间戳Date times','时间差Time deltas','时间段Time spans','日期偏置Date offsets'],
           '单元素类型':['Timestamp','Timedelta','Period','DateOffset'],
           '数组类型':['DatetimeIndex','TimedeltaIndex','PeriodIndex','None'],
           'pandas数据类型':['datetime64[ns]','timedelta64[ns]','period[freq]','None']}
ts_df = pd.DataFrame(ts_dict)
ts_df

,概念,单元素类型,数组类型,pandas数据类型
0,时间戳Date times,Timestamp,DatetimeIndex,datetime64[ns]
1,时间差Time deltas,Timedelta,TimedeltaIndex,timedelta64[ns]
2,时间段Time spans,Period,PeriodIndex,period[freq]
3,日期偏置Date offsets,DateOffset,None,None


## 时间戳
### Timestamp的构造与属性
1. 单个时间戳的生成利用`pd.Timestamp`实现
2. 通过`year/month/day/hour/min/second`可以获取具体的数值
### Datetime序列的生成
1. 一组时间戳可以组成时间序列，可利用`to_datetime`和`date_range`生成，`to_datetime`能够把一列时间戳格式的对象转换为datetime64[ns]类型的时间序列
2. `DatetimeIndex`要转为`datetime64[ns]`的序列，需使用`pd.Series`
3. `to_datetime`还可以把表的多列时间属性拼接转为时间序列
4. `date_range`是一种生成连续间隔时间的方法
   - `start`开始时间
   - `end`结束时间
   - `freq`时间间隔
   - `periods`时间戳个数
   - 四个中的三个参数决定了，剩下的一个就随之决定了，开始或结束日期如果作为端点则会被包含
5. 一种改变序列采样频率的方法`asfreq`，能够根据给定的`freq`对序列进行类似于`reindex`的操作
6. datetime64[ns]本质上可以理解为一个大整数，对于一个该类型的序列，可以使用max，min，mean来取得最大时间戳，最小时间戳和“平均”时间戳
### dt对象
1. 取出时间相关的属性
   - 常用属性包括：date,time,year,month,day,hour,minute,second,microsecond,nanosecond,dayofweek(周中的星期情况，周一为0),dayofyear,weekofyear,daysinmonth(该月一共有几天),quarter(季度)
   - 可以通过month_name,day_name返回英文的月名和星期名，但这是方法不是属性
2. 判断时间戳是否满足条件：是否为月/季/年的第一天或者最后一天
   - is_year_start,is_year_end
3. 取整操作
   - round,ceil,floor，公共参数为freq，常用的包括H,min,S
### 时间戳的切片与索引
1. 利用dt对象和布尔条件联合使用选出某个子时间戳序列
2. 利用切片，常用于连续时间戳

In [4]:
ts = pd.Timestamp('2020/1/1')
ts

Timestamp('2020-01-01 00:00:00')

In [5]:
ts.hour

0

In [2]:
pd.to_datetime(['2020-1-1', '2020-1-3'])

DatetimeIndex(['2020-01-01', '2020-01-03'], dtype='datetime64[ns]', freq=None)

In [3]:
df = pd.read_csv('./data/learn_pandas.csv')
s = pd.to_datetime(df.Test_Date)
s.head()

0   2019-10-05
1   2019-09-04
2   2019-09-12
3   2020-01-03
4   2019-11-06
Name: Test_Date, dtype: datetime64[ns]

In [7]:
temp = pd.to_datetime(['2020--1\\1','2020--1\\3'], format='%Y--%m\\%d')
temp

DatetimeIndex(['2020-01-01', '2020-01-03'], dtype='datetime64[ns]', freq=None)

In [9]:
df_date_cols = pd.DataFrame({'year':[2020,2020],
                             'month':[1,1],
                             'day':[1,2],
                             'hour':[10,20],
                             'minute':[30,50],
                             'second':[20,40]})
pd.to_datetime(df_date_cols)

0   2020-01-01 10:30:20
1   2020-01-02 20:50:40
dtype: datetime64[ns]

In [2]:
pd.date_range('2020-1-1','2020-1-21',freq='10D')

DatetimeIndex(['2020-01-01', '2020-01-11', '2020-01-21'], dtype='datetime64[ns]', freq='10D')

In [3]:
pd.date_range('2020-1-1','2020-2-28',periods=6)

DatetimeIndex(['2020-01-01 00:00:00', '2020-01-12 14:24:00',
               '2020-01-24 04:48:00', '2020-02-04 19:12:00',
               '2020-02-16 09:36:00', '2020-02-28 00:00:00'],
              dtype='datetime64[ns]', freq=None)

In [4]:
s = pd.Series(np.random.rand(5), index = pd.to_datetime(['2020-1-%d'%i for i in range(1,10,2)]))
s

2020-01-01    0.805718
2020-01-03    0.331448
2020-01-05    0.340497
2020-01-07    0.977192
2020-01-09    0.441622
dtype: float64

In [5]:
s.asfreq('D')

2020-01-01    0.805718
2020-01-02         NaN
2020-01-03    0.331448
2020-01-04         NaN
2020-01-05    0.340497
2020-01-06         NaN
2020-01-07    0.977192
2020-01-08         NaN
2020-01-09    0.441622
Freq: D, dtype: float64

In [6]:
s.asfreq('12H').head()

2020-01-01 00:00:00    0.805718
2020-01-01 12:00:00         NaN
2020-01-02 00:00:00         NaN
2020-01-02 12:00:00         NaN
2020-01-03 00:00:00    0.331448
Freq: 12H, dtype: float64

In [3]:
s = pd.Series(pd.date_range('2020-1-1','2020-1-3',freq='D'))
s

0   2020-01-01
1   2020-01-02
2   2020-01-03
dtype: datetime64[ns]

In [5]:
s.dt.quarter

0    1
1    1
2    1
dtype: int64

In [6]:
s.dt.month_name()

0    January
1    January
2    January
dtype: object

In [8]:
s.dt.is_year_start

0     True
1    False
2    False
dtype: bool

In [6]:
s = pd.Series(pd.date_range('2020-1-1 20:16:00', '2020-1-1 22:35:00', freq = '45min'))
s

0   2020-01-01 20:16:00
1   2020-01-01 21:01:00
2   2020-01-01 21:46:00
3   2020-01-01 22:31:00
dtype: datetime64[ns]

In [7]:
s.dt.round('1H') # 四舍五入，超过30分钟则为下一个小时（不包含30）

0   2020-01-01 20:00:00
1   2020-01-01 21:00:00
2   2020-01-01 22:00:00
3   2020-01-01 23:00:00
dtype: datetime64[ns]

In [8]:
s.dt.ceil('1H') # 向上取整

0   2020-01-01 21:00:00
1   2020-01-01 22:00:00
2   2020-01-01 22:00:00
3   2020-01-01 23:00:00
dtype: datetime64[ns]

In [9]:
s.dt.floor('1H') # 向下取整

0   2020-01-01 20:00:00
1   2020-01-01 21:00:00
2   2020-01-01 21:00:00
3   2020-01-01 22:00:00
dtype: datetime64[ns]

In [10]:
s = pd.Series(np.random.randint(2,size=366), index=pd.date_range('2020-01-01','2020-12-31'))
idx = pd.Series(s.index).dt
s.head()

2020-01-01    1
2020-01-02    1
2020-01-03    1
2020-01-04    0
2020-01-05    1
Freq: D, dtype: int32

In [11]:
# 每月的第一天或者最后一天
s[(idx.is_month_start|idx.is_month_end).values].head()

2020-01-01    1
2020-01-31    0
2020-02-01    1
2020-02-29    0
2020-03-01    0
dtype: int32

In [12]:
# 双休日
s[idx.dayofweek.isin([5,6]).values].head()

2020-01-04    0
2020-01-05    1
2020-01-11    0
2020-01-12    1
2020-01-18    0
dtype: int32

In [13]:
# 取出单日值
s['2020-01-01']

1

In [14]:
s['20200101']

1

In [15]:
# 取出七月
s['2020-07'].head()

2020-07-01    1
2020-07-02    1
2020-07-03    1
2020-07-04    1
2020-07-05    0
Freq: D, dtype: int32

In [16]:
# 取出5月初至7月15日
s['2020-05':'2020-7-15'].head()

2020-05-01    1
2020-05-02    0
2020-05-03    0
2020-05-04    0
2020-05-05    1
Freq: D, dtype: int32

## 时间差
### 1.Timedelta的生成
1. 时间差可以理解为两个时间戳的差，可以通过`pd.Timedelta`构造
2. 生成时间差序列的主要方式是pd.to_timedelta，类型为timedelta64[ns]
3. 与date_range一样，时间序列也可用timedelta_range生成，两者具有一致的参数
4. 对于Timedelta序列，同样定义了dt对象，上面主要定义了的属性包括days，seconds，mircroseconds，nanoseconds，其中
   - 这里的seconds不是指单纯的秒，而是**对天数取余后剩余的秒数**
   - 如果不想对天数取余而直接对应秒数，可以使用`total_seconds`
5. 与时间戳序列类似，取整函数也可以在dt对象上使用
### 2.Timedelta的运算
1. 时间差支持的常用运算：
   - 与标量的乘法运算
   - 与时间戳的加减法运算
   - 与时间差的加减法运算
   - 与时间差的除法运算

In [17]:
pd.Timestamp('20200102 08:00:00')-pd.Timestamp('20200101 07:35:00')

Timedelta('1 days 00:25:00')

In [18]:
pd.Timedelta(days=1, minutes=25)

Timedelta('1 days 00:25:00')

In [23]:
# 字符串生成，自由度很高，写d，day，days都能识别
pd.Timedelta('1 d 25 m')

Timedelta('0 days 02:05:00')

In [26]:
df = pd.read_csv('./data/learn_pandas.csv')

In [27]:
s = pd.to_timedelta(df.Time_Record)
s.head()

0   0 days 00:04:34
1   0 days 00:04:20
2   0 days 00:05:22
3   0 days 00:04:08
4   0 days 00:05:22
Name: Time_Record, dtype: timedelta64[ns]

In [28]:
pd.timedelta_range('0s','1000s',freq='6min')

TimedeltaIndex(['0 days 00:00:00', '0 days 00:06:00', '0 days 00:12:00'], dtype='timedelta64[ns]', freq='6T')

In [29]:
pd.timedelta_range('0s','1000s',periods=6)

TimedeltaIndex(['0 days 00:00:00', '0 days 00:03:20', '0 days 00:06:40',
                '0 days 00:10:00', '0 days 00:13:20', '0 days 00:16:40'],
               dtype='timedelta64[ns]', freq=None)

In [30]:
s.dt.seconds.head()

0    274
1    260
2    322
3    248
4    322
Name: Time_Record, dtype: int64

In [31]:
s.dt.total_seconds().head()

0    274.0
1    260.0
2    322.0
3    248.0
4    322.0
Name: Time_Record, dtype: float64

In [32]:
pd.to_timedelta(df.Time_Record).dt.round('min').head()

0   0 days 00:05:00
1   0 days 00:04:00
2   0 days 00:05:00
3   0 days 00:04:00
4   0 days 00:05:00
Name: Time_Record, dtype: timedelta64[ns]

## 日期偏置
### Offset对象
1. 一种和日历相关的特殊时间差
   - 特殊的Offset对象CDay,其中holidays，weekmask参数能够分别对自定义的日期和星期进行过滤，前者传入需要过滤的日期列表，后者传入的是三个字母的星期缩写构成的星期字符串，作用是只保留字符串中出现的星期
2. date_range的freq取值可用Offset对象
   - 月初：'MS',pd.offsets.MonthBegin()
   - 月末：'M',pd.offsets.MonthEnd()
   - 工作日：'B',pd.offsets.BDay()
   - 周一：'W-MON',pd.offsets.CDay(weekmask='Mon')
   - 每月第一个周一：'WOM-1MON',pd.offsets.WeekOfMonth(week=0,weekday=0)

In [34]:
pd.Timestamp('20200831') + pd.offsets.WeekOfMonth(week=0, weekday=0)

Timestamp('2020-09-07 00:00:00')

In [35]:
pd.Timestamp('20200907')+pd.offsets.BDay(30)

Timestamp('2020-10-19 00:00:00')

In [38]:
my_filter = pd.offsets.CDay(n=1, weekmask='Wed Fri', holidays=['20200109'])
dr = pd.date_range('20200108','20200111')
dr.to_series().dt.dayofweek

2020-01-08    2
2020-01-09    3
2020-01-10    4
2020-01-11    5
Freq: D, dtype: int64

In [39]:
[i + my_filter for i in dr]

[Timestamp('2020-01-10 00:00:00'),
 Timestamp('2020-01-10 00:00:00'),
 Timestamp('2020-01-15 00:00:00'),
 Timestamp('2020-01-15 00:00:00')]

## 时序中的滑窗与分组
### 滑动窗口
### 重采样
1. 用法类似groupby